In [1]:
from trajectree.fock_optics.utils import create_vacuum_state
from trajectree.fock_optics.devices import global_phase
from trajectree.quant_info.noise_models import amplitude_damping
from trajectree.quant_info.circuit import Circuit
from trajectree.trajectory import quantum_channel, trajectory_evaluator
import numpy as np
from scipy import sparse as sp
from matplotlib import pyplot as plt
import cirq
import qsimcirq
import time

from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import EstimatorV2 as Estimator
from qiskit import transpile, QuantumCircuit
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit.primitives import SamplerResult
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    amplitude_damping_error,
    thermal_relaxation_error,
)

from mqt.bench import BenchmarkLevel, get_benchmark

In [2]:
qc = Circuit(1, backend = 'tensor')
qc.X_gate(0)
qc.amplitude_damping(0.5, 0, tag = "amplitude_damping")
qc.create_trajectree()
# qc.t_eval.quantum_channels[0].get_ops()
# qc.t_eval.quantum_channels[0].get_ops()
qc.perform_trajectree_simulation(5, error_tolerance = 1e-10)

time taken: 0.011626958847045898
[1.0]
time taken: 0.0010499954223632812
[1.0, 1.0]
time taken: 0.0013175010681152344
[1.0, 1.0, 1.0]
time taken: 0.0009400844573974609
[1.0, 1.0, 1.0, 1.0]
time taken: 0.0011019706726074219
[1.0, 1.0, 1.0, 1.0, 1.0]
done with simulations


([1.0, 1.0, 1.0, 1.0, 1.0],
 [0.011626958847045898,
  0.0010499954223632812,
  0.0013175010681152344,
  0.0009400844573974609,
  0.0011019706726074219])

In [3]:
from qutip import basis, expand_operator, tensor
from qutip_qip.operations import H, CNOT, S, T, X, Z
from qutip import Qobj
from scipy import sparse as sp

num_qubits = 3
op = CNOT(0,1).get_compact_qobj()
# print(type(op))
zero = tensor([basis(2, 0)] * num_qubits)

# X_op = X(0).get_compact_qobj() 
X_sparse = sp.csr_array(np.array([[0,1],[1,0]]))
X_op = Qobj(X_sparse.toarray())
state = expand_operator(oper = X_op, dims = [2] * num_qubits, targets = [0]) * zero

state = expand_operator(oper = op, dims = [2] * num_qubits, targets = [0,1]) * state
# expand_operator(oper = op, dims = [2] * num_qubits, targets = [0,3]) * state

# expand_operator()
phi = state
psi = state
np.real(phi.dag() * psi)



1.0

Benchmarking circuits

In [4]:
num_qubits = 10
num_trajectories = 50
qc = get_benchmark("qnn", BenchmarkLevel.ALG, num_qubits)
qc.draw(output="mpl")

# 'ae', 'bmw_quark_cardinality', 'bmw_quark_copula', 'bv', 'cdkm_ripple_carry_adder', 'dj', 'draper_qft_adder', 'full_adder', 'ghz', 'graphstate', 'grover', 'half_adder', 'hhl', 'hrs_cumulative_multiplier', 'modular_adder', 'multiplier', 'qaoa', 'qft', 'qftentangled', 
# 'qnn', 'qpeexact', 'qpeinexact', 'qwalk', 'randomcircuit', 'rg_qft_multiplier', 'shor', 'vbe_ripple_carry_adder', 'vqe_real_amp', 'vqe_su2', 'vqe_two_local', 'wstate']

# from qiskit import QuantumCircuit
 
# qc = QuantumCircuit(num_qubits)
# for i in range(num_qubits):
#     qc.x(i)
# for i in range(num_qubits):
#     qc.h(i)
# for i in range(1, num_qubits):
#     qc.cx(i-1, i)
qc.draw()

┌───┐┌───────────┐┌───┐┌───────────┐ ┌────────────┐                   »
    q_0: ┤ H ├┤ P(12.013) ├┤ H ├┤ P(12.013) ├─┤ Ry(5.1857) ├───────────────────»
         ├───┤├───────────┤├───┤├───────────┤ ├────────────┤                   »
    q_1: ┤ H ├┤ P(2.6098) ├┤ H ├┤ P(2.6098) ├─┤ Ry(2.1251) ├───────────────────»
         ├───┤├───────────┤├───┤├───────────┤ ├────────────┤                   »
    q_2: ┤ H ├┤ P(10.411) ├┤ H ├┤ P(10.411) ├─┤ Ry(3.6176) ├───────────────────»
         ├───┤├───────────┤├───┤├───────────┤ ├────────────┤                   »
    q_3: ┤ H ├┤ P(1.8759) ├┤ H ├┤ P(1.8759) ├─┤ Ry(4.7331) ├───────────────────»
         ├───┤├───────────┤├───┤├───────────┤ ├────────────┤                   »
    q_4: ┤ H ├┤ P(6.4441) ├┤ H ├┤ P(6.4441) ├─┤ Ry(5.1968) ├───────────────────»
         ├───┤└┬──────────┤├───┤└┬──────────┤ ├───────────┬┘                   »
    q_5: ┤ H ├─┤ P(1.708) ├┤ H ├─┤ P(1.708) ├─┤ Ry(5.865) ├────────────────────»
         ├───┤┌┴──────────┤├───┤┌┴──────────┤┌┴───────────┴┐                   »
    q_6: ┤ H ├┤ P(8.6587) ├┤ H ├┤ P(8.6587) ├┤ Ry(0.91103) ├───────────────────»
         ├───┤├───────────┤├───┤├───────────┤└┬────────────┤                   »
    q_7: ┤ H ├┤ P(10.578) ├┤ H ├┤ P(10.578) ├─┤ Ry(4.6846) ├───────────■───────»
         ├───┤├───────────┤├───┤├───────────┤┌┴────────────┤         ┌─┴─┐     »
    q_8: ┤ H ├┤ P(5.3471) ├┤ H ├┤ P(5.3471) ├┤ Ry(0.87557) ├──■──────┤ X ├─────»
         ├───┤├───────────┤├───┤├───────────┤└┬────────────┤┌─┴─┐┌───┴───┴────┐»
    q_9: ┤ H ├┤ P(12.025) ├┤ H ├┤ P(12.025) ├─┤ Ry(5.6959) ├┤ X ├┤ Ry(4.2641) ├»
         └───┘└───────────┘└───┘└───────────┘ └────────────┘└───┘└────────────┘»
meas: 10/══════════════════════════════════════════════════════════════════════»
                                                                               »
«                                                                 »
«    q_0: ────────────────────────────────────────────────────────»
«                                                                 »
«    q_1: ────────────────────────────────────────────────────────»
«                                                                 »
«    q_2: ────────────────────────────────────────────────────────»
«                                                                 »
«    q_3: ────────────────────────────────────────────────■───────»
«                                                       ┌─┴─┐     »
«    q_4: ──────────────────────────────────■───────────┤ X ├─────»
«                                         ┌─┴─┐     ┌───┴───┴────┐»
«    q_5: ────────────────────■───────────┤ X ├─────┤ Ry(2.0262) ├»
«                           ┌─┴─┐     ┌───┴───┴────┐└────────────┘»
«    q_6: ──────■───────────┤ X ├─────┤ Ry(1.7746) ├──────────────»
«             ┌─┴─┐     ┌───┴───┴────┐└────────────┘              »
«    q_7: ────┤ X ├─────┤ Ry(3.8068) ├────────────────────────────»
«         ┌───┴───┴────┐└────────────┘                            »
«    q_8: ┤ Ry(2.0971) ├──────────────────────────────────────────»
«         └────────────┘                                          »
«    q_9: ────────────────────────────────────────────────────────»
«                                                                 »
«meas: 10/════════════════════════════════════════════════════════»
«                                                                 »
«                                                   ┌────────────┐ ░ ┌─┐      »
«    q_0: ──────────────────────────────────■───────┤ Ry(1.4207) ├─░─┤M├──────»
«                                         ┌─┴─┐     ├────────────┤ ░ └╥┘┌─┐   »
«    q_1: ────────────────────■───────────┤ X ├─────┤ Ry(5.3611) ├─░──╫─┤M├───»
«                           ┌─┴─┐     ┌───┴───┴────┐└────────────┘ ░  ║ └╥┘┌─┐»
«    q_2: ──────■───────────┤ X ├─────┤ Ry(1.9247) ├───────────────░──╫──╫─┤M├»
«             ┌─┴─┐     ┌───┴───┴────┐└────────────┘               ░  ║  ║ └╥┘»
«    q_3: ────┤ X ├───

In [5]:
# print(dumps(qc))
target_basis = ['h', 's', 'x', 't', 'ry', 'rz', 'cx', 'cz']
transpiled_qc_custom = transpile(qc, basis_gates=target_basis, optimization_level=1)

noise_probability = 0.01
noise_channel = amplitude_damping_error(noise_probability)

two_qubit_noise_channel = noise_channel.tensor(noise_channel)

noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(noise_channel, target_basis[:-2])
noise_model.add_all_qubit_quantum_error(two_qubit_noise_channel, ["cx", 'cz'])

print(noise_model)
transpiled_qc_custom.draw(output="text")
# noisy_transpiled_qc_custom = QuantumCircuit(transpiled_qc_custom.num_qubits, transpiled_qc_custom.num_clbits)
# for instr, qargs, cargs in transpiled_qc_custom.data:
#     if instr.name not in {"measure", "barrier"}:
#         noisy_transpiled_qc_custom.append(instr, qargs, cargs)
#         for q in qargs:
#             noisy_transpiled_qc_custom.append(noise_channel, [q])        

# transpiled_qc_custom = noisy_transpiled_qc_custom

# transpiled_qc_custom.draw(output="mpl")

# # for i in transpiled_qc_custom:
# #     print(i.operation.name)
# #     print([qubit._index for qubit in i.qubits])
# # transpiled_qc_custom.draw(output="mpl")

NoiseModel:
  Basis gates: ['cx', 'cz', 'h', 'id', 'ry', 'rz', 's', 'sx', 't', 'x']
  Instructions with noise: ['cx', 'rz', 'x', 't', 'ry', 'cz', 'h', 's']
  All-qubits errors: ['h', 's', 'x', 't', 'ry', 'rz', 'cx', 'cz']


global phase: 2.5554
          ┌────────────┐ ┌─────────────┐ ┌────────────┐                    »
    q_0: ─┤ Rz(2.9355) ├─┤ Ry(0.88486) ├─┤ Rz(2.5259) ├────────────────────»
         ┌┴────────────┤ └┬────────────┤ ├────────────┤                    »
    q_1: ┤ Rz(-2.4249) ├──┤ Ry(1.3336) ├─┤ Rz(2.6752) ├────────────────────»
         └┬────────────┤ ┌┴────────────┤┌┴────────────┤                    »
    q_2: ─┤ Rz(1.1254) ├─┤ Ry(0.62778) ├┤ Rz(-2.2408) ├────────────────────»
         ┌┴────────────┤ ├─────────────┤├─────────────┤                    »
    q_3: ┤ Rz(0.79514) ├─┤ Ry(0.44336) ├┤ Rz(0.73149) ├────────────────────»
         ├─────────────┤ └┬────────────┤├─────────────┤                    »
    q_4: ┤ Rz(-3.0669) ├──┤ Ry(1.0674) ├┤ Rz(-2.9601) ├────────────────────»
         └┬───────────┬┘  ├────────────┤├─────────────┤                    »
    q_5: ─┤ Rz(1.513) ├───┤ Ry(1.2937) ├┤ Rz(0.14135) ├────────────────────»
          ├───────────┴┐  ├────────────┤└┬────────────┤                    »
    q_6: ─┤ Rz(3.0886) ├──┤ Ry(2.5349) ├─┤ Rz(2.0736) ├────────────────────»
         ┌┴────────────┴┐┌┴────────────┤┌┴────────────┴┐                   »
    q_7: ┤ Rz(-0.70531) ├┤ Ry(0.56186) ├┤ Rz(-0.77005) ├───────────■───────»
         ├─────────────┬┘└┬────────────┤└┬────────────┬┘         ┌─┴─┐     »
    q_8: ┤ Rz(-1.0944) ├──┤ Ry(1.6892) ├─┤ Rz(0.5016) ├───■──────┤ X ├─────»
         ├─────────────┤ ┌┴────────────┤ ├────────────┤ ┌─┴─┐┌───┴───┴────┐»
    q_9: ┤ Rz(-2.7716) ├─┤ Ry(0.53446) ├─┤ Rz(2.0926) ├─┤ X ├┤ Ry(4.2641) ├»
         └─────────────┘ └─────────────┘ └────────────┘ └───┘└────────────┘»
meas: 10/══════════════════════════════════════════════════════════════════»
                                                                           »
«                                                                 »
«    q_0: ────────────────────────────────────────────────────────»
«                                                                 »
«    q_1: ────────────────────────────────────────────────────────»
«                                                                 »
«    q_2: ────────────────────────────────────────────────────────»
«                                                                 »
«    q_3: ────────────────────────────────────────────────■───────»
«                                                       ┌─┴─┐     »
«    q_4: ──────────────────────────────────■───────────┤ X ├─────»
«                                         ┌─┴─┐     ┌───┴───┴────┐»
«    q_5: ────────────────────■───────────┤ X ├─────┤ Ry(2.0262) ├»
«                           ┌─┴─┐     ┌───┴───┴────┐└────────────┘»
«    q_6: ──────■───────────┤ X ├─────┤ Ry(1.7746) ├──────────────»
«             ┌─┴─┐     ┌───┴───┴────┐└────────────┘              »
«    q_7: ────┤ X ├─────┤ Ry(3.8068) ├────────────────────────────»
«         ┌───┴───┴────┐└────────────┘                            »
«    q_8: ┤ Ry(2.0971) ├──────────────────────────────────────────»
«         └────────────┘                                          »
«    q_9: ────────────────────────────────────────────────────────»
«                                                                 »
«meas: 10/════════════════════════════════════════════════════════»
«                                                                 »
«                                                   ┌────────────┐ ░ ┌─┐      »
«    q_0: ──────────────────────────────────■───────┤ Ry(1.4207) ├─░─┤M├──────»
«                                         ┌─┴─┐     ├────────────┤ ░ └╥┘┌─┐   »
«    q_1: ────────────────────■───────────┤ X ├─────┤ Ry(5.3611) ├─░──╫─┤M├───»
«                           ┌─┴─┐     ┌───┴───┴────┐└────────────┘ ░  ║ └╥┘┌─┐»
«    q_2: ──────■───────────┤ X ├─────┤ Ry(1.9247) ├───────────────░──╫──╫─┤M├»
«             ┌─┴─┐     ┌───┴───┴────┐└────────────┘               ░  ║  ║ └╥┘»
«    q_3: ────┤ X ├─────┤ Ry(6.0936) ├─────────────────────────────░──╫──╫──╫─»
«   

In [6]:
observable = SparsePauliOp("Z" * (num_qubits))

# Use the Sampler primitive for latest API
backend = AerSampler()

noisy_estimator = Estimator(
    options=dict(backend_options=dict(noise_model=noise_model), run_options=dict(shots=num_trajectories))
)

pub = (transpiled_qc_custom, observable)

job = noisy_estimator.run([pub])
result = job.result()
pub_result = result[0]
# print((len(result)))
print(float(pub_result.data.evs))
print(float(pub_result.data.stds))


# estimator = Estimator()
# estimator.options.run_options = {"shots":10000}
# estimator.options.backend_options = dict(noise_model=noise_channel)
# # print(estimator.options)

# # Run the sampler
# job = estimator.run([(transpiled_qc_custom, observable)])
# result = job.result()

# pub_result = result[0]
# print(result)
# print(pub_result.data.stds)
# print(pub_result.data.evs)

0.16
0.0


In [7]:
import cirq
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"


# Create Cirq qubits
num_qubits_cirq = transpiled_qc_custom.num_qubits
cirq_qubits = cirq.LineQubit.range(num_qubits_cirq)

# Map Qiskit gates to Cirq gates
gate_map = {
    'h': lambda q: cirq.H(q),
    's': lambda q: cirq.S(q),
    't': lambda q: cirq.T(q),
    'x': lambda q: cirq.X(q),
    'ry': lambda theta, q: cirq.ry(theta).on(q),
    'rz': lambda theta, q: cirq.rz(theta).on(q),
    'cx': lambda q0, q1: cirq.CNOT(q0, q1),
    'cz': lambda q0, q1: cirq.CZ(q0, q1),
    'measure': lambda q: None,  # We'll handle measurements separately
    'barrier': lambda *args: None,
}

# Build the Cirq circuit
cirq_circuit = cirq.Circuit()

for circuit_instr in transpiled_qc_custom.data:
    instr = circuit_instr.operation
    qargs = circuit_instr.qubits
    cargs = circuit_instr.clbits
    gate_name = instr.name
    print(instr)
    # if gate_name == 'quantum_channel':
    #     # This is the amplitude damping error
    #     # Extract the damping parameter from the Kraus operators
    #     for qarg in qargs:
    #         cirq_qubit = cirq_qubits[qarg._index]
    #         cirq_circuit.append(cirq.amplitude_damp(gamma=noise_probability)(cirq_qubit))
    
    if gate_name in gate_map and gate_name not in ['measure', 'barrier']:
        cirq_qubit_args = [cirq_qubits[q._index] for q in qargs]
        if gate_name == 'ry' or gate_name == 'rz':
            gate = gate_map[gate_name](instr.params[0], cirq_qubit_args[0])
        else:
            gate = gate_map[gate_name](*cirq_qubit_args)
        
        if gate is not None:
            cirq_circuit.append(gate)
            for qarg in qargs:
                cirq_qubit = cirq_qubits[qarg._index]
                cirq_circuit.append(cirq.amplitude_damp(gamma=noise_probability)(cirq_qubit))

# # Add measurements at the end
# cirq_circuit.append(cirq.measure(*cirq_qubits, key='result'))

print(f"Converted Cirq circuit with {len(cirq_circuit)} moments")
print(f"Number of operations: {len(list(cirq_circuit.all_operations()))}")
print("\nFirst 50 operations:")
for i, op in enumerate(cirq_circuit.all_operations()):
    if i >= 50:
        break
    print(f"{i}: {op}")

# Draw a portion of the circuit
print("\nCircuit diagram (first 20 moments):")
print(cirq.Circuit(list(cirq_circuit)))


Instruction(name='rz', num_qubits=1, num_clbits=0, params=[2.9354624744748357])
Instruction(name='ry', num_qubits=1, num_clbits=0, params=[0.8848555776377272])
Instruction(name='rz', num_qubits=1, num_clbits=0, params=[2.5258852330273633])
Instruction(name='rz', num_qubits=1, num_clbits=0, params=[-2.4248945199862293])
Instruction(name='ry', num_qubits=1, num_clbits=0, params=[1.3335639076683454])
Instruction(name='rz', num_qubits=1, num_clbits=0, params=[2.675228581171697])
Instruction(name='rz', num_qubits=1, num_clbits=0, params=[1.1253758942803103])
Instruction(name='ry', num_qubits=1, num_clbits=0, params=[0.6277814240371993])
Instruction(name='rz', num_qubits=1, num_clbits=0, params=[-2.240773328821211])
Instruction(name='rz', num_qubits=1, num_clbits=0, params=[0.7951425403628463])
Instruction(name='ry', num_qubits=1, num_clbits=0, params=[0.44335665557274084])
Instruction(name='rz', num_qubits=1, num_clbits=0, params=[0.7314939130866542])
Instruction(name='rz', num_qubits=1, nu

In [8]:
# Set the "noisy repetitions" to 100.
# This parameter only affects expectation value calculations.
options = {'r': num_trajectories}
# Also set the random seed to get reproducible results.
ev_simulator = qsimcirq.QSimSimulator(qsim_options=options)
# Define observables to measure: <Z> for q0 and <X> for q1.
observable = cirq.Z(cirq_qubits[0])
for i in range(1,num_qubits):
    observable *= cirq.Z(cirq_qubits[i])
# Calculate expectation values for the given observables.
start_time = time.time()
ev_results = ev_simulator.simulate_expectation_values(
    cirq_circuit,
    observables=[observable],
)
print("elapsed time:", time.time() - start_time)
print(ev_results)

elapsed time: 0.017868518829345703
[(-0.001974298943928261+2.3882180761529253e-10j)]


## Now, we implement the same circuit in Trajectree

In [9]:
qc = Circuit(num_qubits, backend = 'tensor')
qc.qiskit_to_trajectree(transpiled_qc_custom, noise_parameter=noise_probability)
qc.expectation('z'*(num_qubits))


In [ ]:
evs, times = qc.perform_trajectree_simulation(num_trajectories)
print("outside of simulation")
# np.mean(evs)
# evs

time taken: 0.9867062568664551
[-0.0025700598339905645]
time taken: 0.00895547866821289
[-0.0025700598339905645, -0.0025700598339905645]
time taken: 0.31093549728393555
[-0.0025700598339905645, -0.0025700598339905645, -0.004111483326230288]
time taken: 0.3929555416107178
[-0.0025700598339905645, -0.0025700598339905645, -0.004111483326230288, -0.0016706982052486095]
time taken: 0.010951757431030273
[-0.0025700598339905645, -0.0025700598339905645, -0.004111483326230288, -0.0016706982052486095, -0.0025700598339905645]
time taken: 0.359835147857666
[-0.0025700598339905645, -0.0025700598339905645, -0.004111483326230288, -0.0016706982052486095, -0.0025700598339905645, 0.007358328965339324]
time taken: 0.011823415756225586
[-0.0025700598339905645, -0.0025700598339905645, -0.004111483326230288, -0.0016706982052486095, -0.0025700598339905645, 0.007358328965339324, -0.0025700598339905645]
time taken: 0.007417917251586914
[-0.0025700598339905645, -0.0025700598339905645, -0.004111483326230288, -0.

[-0.0025700598339905645,
 -0.0025700598339905645,
 -0.004111483326230288,
 -0.0016706982052486095,
 -0.0025700598339905645,
 0.007358328965339324,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.004111483326230288,
 -0.0025700598339905645,
 0.015820827615731958,
 -0.0025700598339905645,
 -0.004111483326230288,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 np.float64(-0.00035977129862499284),
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.0025700598339905645,
 -0.002570059833990

In [11]:
np.mean(evs)

np.float64(-0.0020226899753857347)

In [ ]:
print(len(qc.t_eval.quantum_channels))

110


In [ ]:
qc = Circuit(3)
qc.H_gate(0)
qc.CNOT_gate(0, 2)
qc.create_trajectree()
qc.expectation('xxz')
qc.perform_trajectree_simulation(1)[0][0]

np.complex128(0j)

In [ ]:
def run_experiment(max_cache_nodes=-1):
    num_simulations = 100 # 20

    num_qubits = 10

    cache_size = 1
    iter = 0
    max_iter = 1
    times = []

    while iter < max_iter:  # This while loop is just to perform the entire exoeriment multiple times to get average runtime values.
        # psi, t_eval = generate_test_circuit_trajectree(cache_size, max_cache_nodes)
        qc = Circuit(num_qubits)
        
        for i in range(num_qubits):
            qc.H_gate(i)

        # Damping layer
        for i in range(num_qubits):
            qc.amplitude_damping(0.1, i)

        qc.create_trajectree()

        times_iter = qc.perform_trajectree_simulation(num_simulations)

        times.append(times_iter)
        
        iter += 1
        print("iter:", iter)

    times_avg = np.mean(np.array(times).T, axis = 1)

    avg_times = [np.mean(times_avg[:i]) for i in range(1, len(times_avg))]

    return qc
    # return times
run_experiment()

iter: 1


AttributeError: 'float' object has no attribute 'site_tags'